In [ ]:
%pip install dist/codeflare_sdk-0.0.0.dev0-py3-none-any.whl --force-reinstall

Both of the below examples use a python script that uses functions from a local helper file to demonstrate the ability to store multiple scripts in the config maps.

In [ ]:
from codeflare_sdk import Cluster, ClusterConfiguration, TokenAuthentication, RayJob, ManagedClusterConfig

## Existing Cluster

In [ ]:
# This only works once the cluster's pods have been deleted and recreated as the existing pods won't pick up the newly added config map. Need to resolve this.

cluster = Cluster(ClusterConfiguration(
    name='rayjob-cluster',
    head_extended_resource_requests={'nvidia.com/gpu':0},
    worker_extended_resource_requests={'nvidia.com/gpu':0},
    num_workers=2,
    worker_cpu_requests=2,
    worker_cpu_limits=2,
    worker_memory_requests=2,
    worker_memory_limits=4,
    namespace='rhods-notebooks'
))

cluster.apply()

In [ ]:
cluster.status()

In [ ]:
rayjob = RayJob(
    job_name="sdk-test-job",
    cluster_name="rayjob-cluster",
    namespace="rhods-notebooks",
    entrypoint="python test.py",
)

rayjob.submit()

## New Cluster

In [ ]:
cluster_config = ManagedClusterConfig(
    num_workers=2,
    worker_cpu_requests=1,
    worker_cpu_limits=1,
    worker_memory_requests=4,
    worker_memory_limits=4,
    head_accelerators={'nvidia.com/gpu': 0},
    worker_accelerators={'nvidia.com/gpu': 0},
)

In [ ]:
rayjob = RayJob(
    job_name="sdk-test-job",
    cluster_config=cluster_config,
    namespace="rhods-notebooks",
    entrypoint = 'python -c "import ray; import time; ray.init(); time.sleep(5); ray.shutdown()"'
)

rayjob.submit()

In [ ]:
rayjob.status()